### Properties of Natural Gases
t = temperature of interest (degrees F) \
p = pressure of interest (psia) \
sg = gas specific gravity (air=1) \
n2 = Nitrogen fraction \
co2 = CO2 fraction \
h2s = H2S fraction

In [15]:
import numpy as np
import math
from scipy.integrate import quad

In [2]:
# Calculation of pseudo critical Temperature for Natural Gases, units in deg R (McCain Petroleum Fluids, pp. 120, 512)
def Tpc(sg, co2 = 0, h2s = 0):
    a_fac = (co2 + h2s)
    tc_adj = 120 * ((a_fac ** 0.9) - (a_fac ** 1.6)) + 15 * ((h2s ** 0.5) * (h2s ** 4))
    Tpc1 = 169.2 + 349.5 * sg - 74.0 * (sg ** 2)
    Tpc = Tpc1 - tc_adj
    return Tpc

In [3]:
# Calculation of pseudo critical Pressure for Natural Gases, units in psia (McCain Petroleum Fluids, pp. 120, 512)
def Ppc(sg, co2 = 0, h2s = 0):
    a_fac = (co2 + h2s)
    tc_adj = 120 * ((a_fac ** 0.9) - (a_fac ** 1.6)) + 15 * ((h2s ** 0.5) * (h2s ** 4))
    Tpc1 = 169.2 + 349.5 * sg - 74.0 * (sg ** 2)
    Ppc1 = 756.8 - 131.0 * sg - 3.6 * (sg ** 2)
    Ppc = (Ppc1 * Tpc(sg, co2, h2s)) / (Tpc1 + h2s * (1 - h2s) * tc_adj)
    return Ppc

In [4]:
# Calculation of z-factor, unitless (McCain Petroleum Fluids, pp. 111, 510)
def z_factor(p, t, sg, co2 = 0, h2s = 0):
    a1 = 0.3265
    a2 = -1.07
    a3 = -0.5339
    a4 = 0.01569
    a5 = -0.05165
    a6 = 0.5475
    a7 = -0.7361
    a8 = 0.1844
    a9 = 0.1056
    a10 = 0.6134
    a11 = 0.7210
    Tpr = (t + 460) / Tpc(sg, co2, h2s)
    Ppr = p / Ppc(sg, co2, h2s)
    Rpr = 0.27 * (Ppr / Tpr)
    for i in range(1000):
        z1 = 1 + ((a1 + (a2 / Tpr) + (a3 / (Tpr ** 3)) + (a4 / (Tpr ** 4)) + (a5 / (Tpr ** 5))) * Rpr)
        z2 = (a6 + (a7 / Tpr) + (a8 / (Tpr ** 2))) * (Rpr ** 2)
        z3 = -a9 * ((a7 / Tpr) + (a8 / (Tpr ** 2))) * (Rpr ** 5)
        z4 = a10 * (1 + (a11 * (Rpr ** 2))) * ((Rpr **2) / (Tpr ** 3)) * np.exp(-a11 * (Rpr ** 2))
        z_factor = z1 + z2 + z3 + z4
        Rpr_old = Rpr
        Rpr = 0.27 * (Ppr / (z_factor * Tpr))
        test = abs((Rpr - Rpr_old) / Rpr)
        if test < 0.00001:
            break
    return z_factor


In [5]:
# Calculation of compressibility of natural gas, units of 1/psia (McCain Petroleum Fluids, pp. 176, 512 - 513)
def c_gas(p, t, sg, co2 = 0, h2s = 0):
    a1 = 0.3265
    a2 = -1.07
    a3 = -0.5339
    a4 = 0.01569
    a5 = -0.05165
    a6 = 0.5475
    a7 = -0.7361
    a8 = 0.1844
    a9 = 0.1056
    a10 = 0.6134
    a11 = 0.7210
    Tpr = (t + 460) / Tpc(sg, co2, h2s)
    Ppr = p / Ppc(sg, co2, h2s)
    z = z_factor(p, t, sg, co2, h2s)
    Rpr = 0.27 * (Ppr / (z *Tpr))
    dcpr1 = a1 + (a2 / Tpr) + (a3 / (Tpr ** 3)) + (a4 / (Tpr ** 4)) + (a5 / (Tpr ** 5))
    dcpr2 = 2 * Rpr * (a6 + (a7 / Tpr) + (a8 / (Tpr ** 2))) - 5 * (Rpr ** 4) * a9 * ((a7 / Tpr) + (a8 / (Tpr ** 2)))
    dcpr3 = ((2 * a10 * Rpr) / (Tpr ** 3)) * (1 + (a11 * (Rpr ** 2)) - ((a11 ** 2) * (Rpr ** 4))) * np.exp(-a11 * (Rpr ** 2))
    dcpr = dcpr1 + dcpr2 + dcpr3
    cpr = (1 / Ppr) - (0.27 / ((z ** 2) * Tpr)) * (dcpr / (1 + (Rpr / z) * dcpr))
    cg = cpr / Ppc(sg, co2, h2s)
    return cg

In [6]:
# Calculation of viscosity of natural gas, units of cp (McCain Petroleum Fluids, pp. 105, 514 - 515)
def mu_gas(p, t, sg, co2 = 0, h2s = 0):
    z = z_factor(p, t, sg, co2, h2s)
    mw = 28.97 * sg
    R = 10.732 # units of (psia * cu-ft) / (lb-mole * deg R)
    rhog = ((p * mw) / (z * R * (t + 460))) / (62.427962) # units of gm/cc
    mu_afac = ((9.379 + 0.01607 * mw) * ((t + 460) ** 1.5)) / (209.2 + 19.26 * mw + (t + 460))
    mu_bfac = 3.448 + (986.4 / (t + 460)) + 0.01009 * mw
    mu_cfac = 2.447 - 0.2224 * mu_bfac
    mu_g = (mu_afac / 10000) * np.exp(mu_bfac * (rhog ** mu_cfac))
    return mu_g

In [7]:
# Calculation of gas formation volume factor, units of cu-ft - scf (Craft & Hawkins Applied Petroleum Reservoir Engineering, pg. 23)
def b_gas(p, t, sg, co2 = 0, h2s = 0):
    z = z_factor(p, t, sg, co2, h2s)
    bg = 0.02829 * (z * (t + 460)) / p
    return bg

In [32]:
# Calculation of gas pseudo pressure (Petroleum Engineering Handbook Va, pg 757)
def mp_gas(p, t, sg, co2 = 0, h2s = 0):
    def integrand(p):
        mu_g = mu_gas(p, t, sg, co2, h2s)
        z = z_factor(p, t, sg, co2, h2s)
        integrand = p / (mu_g * z)
        return integrand
    mp = quad(integrand, 0, p)[0]
    return mp